In [1]:
import geopandas as gpd
import numpy as np
import shapely.geometry
from shapely.geometry import Point
import shapely.ops
import shapely
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

# geostl nghbd codes:
    # gravois park = 19
    # benton park west = 30
    # dutchtown = 16
    # mount pleasant = 17

In [2]:
nibrs_df = pd.read_csv('crime_data/nibrs_thru_dec_2023.csv')
nibrs_df

C:\Users\diper\AppData\Local\Temp\ipykernel_2748\382449808.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  nibrs_df = pd.read_csv('crime_data/nibrs_thru_dec_2023.csv')


,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip
0,21000002,1/1/2021,02:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,422,4.0,36,38.629573,-90.204995,1700 CHESTNUT ST,NaN,Saint Louis,MO,NaN
1,21000003,1/1/2021,09:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0
2,21000003,1/1/2021,09:00.0,PROPERTY DAMAGE,A,290,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0
3,21000007,1/1/2021,10:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,223,2.0,9,38.598725,-90.310722,6963 PERNOD AVE,NaN,Saint Louis,MO,63139.0
4,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173359,24000551,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,CITY1,1.0,16,38.587915,-90.252870,3920 DUNNICA AVE,NaN,Saint Louis,MO,NaN
173360,24000557,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,627,6.0,68,38.677708,-90.217160,2007 E FAIR AVE,NaN,Saint Louis,MO,NaN
173361,24000591,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,127,1.0,3,38.569796,-90.254490,5516 GRACE AVE,NaN,Saint Louis,MO,NaN
173362,24000629,12/31/2023,00:00.0,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,A,23G,CITY4,4.0,36,38.630687,-90.201630,1500 OLIVE ST,NaN,Saint Louis,MO,NaN


In [39]:
crime_list = sorted(nibrs_df['offense'].unique())

In [3]:
nibrs_geo = [Point(xy) for xy in zip(nibrs_df['longitude'], nibrs_df['latitude'])]
nibrs_df = gpd.GeoDataFrame(nibrs_df, geometry=nibrs_geo)
nibrs_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
0,21000002,1/1/2021,02:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,422,4.0,36,38.629573,-90.204995,1700 CHESTNUT ST,NaN,Saint Louis,MO,NaN,POINT (-90.20499 38.62957)
1,21000003,1/1/2021,09:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0,POINT (-90.26423 38.64122)
2,21000003,1/1/2021,09:00.0,PROPERTY DAMAGE,A,290,521,5.0,38,38.641221,-90.264226,4961 LACLEDE AVE,210,Saint Louis,MO,63108.0,POINT (-90.26423 38.64122)
3,21000007,1/1/2021,10:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,223,2.0,9,38.598725,-90.310722,6963 PERNOD AVE,NaN,Saint Louis,MO,63139.0,POINT (-90.31072 38.59873)
4,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173359,24000551,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,CITY1,1.0,16,38.587915,-90.252870,3920 DUNNICA AVE,NaN,Saint Louis,MO,NaN,POINT (-90.25287 38.58792)
173360,24000557,12/31/2023,00:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,627,6.0,68,38.677708,-90.217160,2007 E FAIR AVE,NaN,Saint Louis,MO,NaN,POINT (-90.21716 38.67771)
173361,24000591,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,127,1.0,3,38.569796,-90.254490,5516 GRACE AVE,NaN,Saint Louis,MO,NaN,POINT (-90.25449 38.56980)
173362,24000629,12/31/2023,00:00.0,STEALING UNDER $150 (PETTY LARCENY-MOTOR VEH P...,A,23G,CITY4,4.0,36,38.630687,-90.201630,1500 OLIVE ST,NaN,Saint Louis,MO,NaN,POINT (-90.20163 38.63069)


In [4]:
nghbrhd_list = ['16', '17', '19', '30']
mask = nibrs_df['neighborhood'].isin(nghbrhd_list)
nbhd_nibrs_df = nibrs_df[mask]
nbhd_nibrs_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
4,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
5,21000008,1/1/2021,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
11,21000032,1/1/2021,52:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,127,1.0,16,38.571658,-90.251776,S GRAND BLVD,LIERMANN AVE,Saint Louis,MO,63116.0,POINT (-90.25178 38.57166)
37,21000075,1/1/2021,37:00.0,RESISTING ARREST/DETENTION/STOP BY FLEEING - C...,B,90Z,322,3.0,16,38.588583,-90.244290,CHIPPEWA ST,S GRAND BLVD,Saint Louis,MO,NaN,POINT (-90.24429 38.58858)
38,21000077,1/1/2021,20:00.0,BURGLARY - 2ND DEGREE,A,220,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173311,23059360,12/31/2023,30:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,127,1.0,16,38.584574,-90.250810,3891 MERAMEC ST,203,Saint Louis,MO,NaN,POINT (-90.25081 38.58457)
173312,23059365,12/31/2023,11:00.0,POSSESSION OF CONTROLLED SUBSTANCE,A,35A,322,3.0,16,38.581622,-90.243918,4130 S. GRAND BOULEVARD,NaN,Saint Louis,MO,63118,POINT (-90.24392 38.58162)
173315,23059375,12/31/2023,45:00.0,ASSAULT 1ST DEGREE OR ATTEMPT,A,13A,323,3.0,19,38.588943,-90.239015,3716 VIRGINIA AVE,NaN,Saint Louis,MO,63118,POINT (-90.23901 38.58894)
173336,24000111,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,125,1.0,17,38.572195,-90.235080,4625 OREGON AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23508 38.57220)


In [15]:
bpw_gp_df = nbhd_nibrs_df[(nbhd_nibrs_df['neighborhood'] == '19') | (nbhd_nibrs_df['neighborhood'] == '30')].reset_index(drop=True)
bpw_gp_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
0,21000077,1/1/2021,20:00.0,BURGLARY - 2ND DEGREE,A,220,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
1,21000077,1/1/2021,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,321,3.0,19,38.594003,-90.229134,2733 CHEROKEE ST,NaN,Saint Louis,MO,63118.0,POINT (-90.22913 38.59400)
2,21015001,1/1/2021,01:00.0,STATUTORY SODOMY OR ATTEMPTED STATUTORY SODOMY...,A,11B,321,3.0,19,38.591852,-90.230130,3500 CALIFORNIA AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23013 38.59185)
3,21038699,1/1/2021,01:00.0,IDENTITY THEFT OR ATTEMPT,A,26F,323,3.0,19,38.589626,-90.238146,3653 S COMPTON AVE,NaN,Saint Louis,MO,63118.0,POINT (-90.23815 38.58963)
4,21055455,1/1/2021,01:00.0,CHILD MOLESTATION - 2ND DEGREE -CHILD LESS THA...,A,11D,325,3.0,30,38.597826,-90.234380,3108 WYOMING ST,NaN,Saint Louis,MO,NaN,POINT (-90.23438 38.59783)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,23059139,12/30/2023,45:00.0,DISCHARGING FIREARM WITHIN CITY (SHOTSPOTTER),A,520,323,3.0,19,38.593012,-90.238444,3300 POTOMAC AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23844 38.59301)
5193,23059217,12/30/2023,50:00.0,TRESPASSING ON PRIVATE PROPERTY,B,90J,321,3.0,19,38.590583,-90.228837,2711 MIAMI,NaN,Saint Louis,MO,63118,POINT (-90.22884 38.59058)
5194,23059231,12/30/2023,49:00.0,DESTRUCTION OF PRIVATE PROPERTY (CITY CHARGE),A,290,325,3.0,30,38.601067,-90.225990,2916 OHIO AVE,NaN,Saint Louis,MO,63118,POINT (-90.22599 38.60107)
5195,23059308,12/31/2023,01:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,325,3.0,30,38.601758,-90.230450,GRAVOIS AVE,CALIFORNIA,Saint Louis,MO,NaN,POINT (-90.23045 38.60176)


In [14]:
dt_mp_df = nbhd_nibrs_df[(nbhd_nibrs_df['neighborhood'] == '16') | (nbhd_nibrs_df['neighborhood'] == '17')].reset_index(drop=True)
dt_mp_df

,inci_id,date_inc,time_inc,offense,nibrs_grp,nibrs_code,beat,district,neighborhood,latitude,longitude,incidentlocation,locationStreet2_Apt,city,state,zip,geometry
0,21000008,1/1/2021,20:00.0,MISCELLANEOUS WEAPON VIOLATION,A,520,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
1,21000008,1/1/2021,20:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,126,1.0,16,38.579494,-90.243210,3500 KINGSLAND CT,NaN,Saint Louis,MO,NaN,POINT (-90.24321 38.57949)
2,21000032,1/1/2021,52:00.0,LEAVING THE SCENE OF A MOTOR VEHICLE ACCIDENT,B,90Z,127,1.0,16,38.571658,-90.251776,S GRAND BLVD,LIERMANN AVE,Saint Louis,MO,63116.0,POINT (-90.25178 38.57166)
3,21000075,1/1/2021,37:00.0,RESISTING ARREST/DETENTION/STOP BY FLEEING - C...,B,90Z,322,3.0,16,38.588583,-90.244290,CHIPPEWA ST,S GRAND BLVD,Saint Louis,MO,NaN,POINT (-90.24429 38.58858)
4,21000112,1/1/2021,30:00.0,TRESPASS,B,90J,127,1.0,16,38.587069,-90.249287,3761 KEOKUK ST,NaN,Saint Louis,MO,63111.0,POINT (-90.24929 38.58707)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11482,23059331,12/31/2023,00:00.0,DOMESTIC ASSAULT - 2ND DEGREE,A,13B,127,1.0,16,38.587053,-90.249017,3747 KEOKUK ST,NaN,Saint Louis,MO,NaN,POINT (-90.24902 38.58705)
11483,23059360,12/31/2023,30:00.0,PROPERTY DAMAGE - 2ND DEGREE,A,290,127,1.0,16,38.584574,-90.250810,3891 MERAMEC ST,203,Saint Louis,MO,NaN,POINT (-90.25081 38.58457)
11484,23059365,12/31/2023,11:00.0,POSSESSION OF CONTROLLED SUBSTANCE,A,35A,322,3.0,16,38.581622,-90.243918,4130 S. GRAND BOULEVARD,NaN,Saint Louis,MO,63118,POINT (-90.24392 38.58162)
11485,24000111,12/31/2023,00:00.0,DISCHARGING FIREARM WITHIN CITY,A,520,125,1.0,17,38.572195,-90.235080,4625 OREGON AVE,NaN,Saint Louis,MO,NaN,POINT (-90.23508 38.57220)


In [18]:
gj_shp = gpd.read_file("gj_area.shx")
gj_shp

,geometry
0,"POLYGON ((-90.22300 38.60493, -90.22799 38.581..."


In [20]:
dt_mp_df['geometry'].within(gj_shp.loc[0, 'geometry']).value_counts()

False    7931
True     3556
Name: count, dtype: int64

In [22]:
geo_mask = dt_mp_df['geometry'].within(gj_shp.loc[0, 'geometry'])
dt_mp_df = dt_mp_df[geo_mask].reset_index(drop=True)

### Here is where i have both dfs and need to combine by month and get the crime counts

In [27]:
dec_dt_mp = dt_mp_df[dt_mp_df['date_inc'].str.contains('12/.*/2023', regex=True)]
dec_bpw_gp = bpw_gp_df[bpw_gp_df['date_inc'].str.contains('12/.*/2023', regex=True)]

In [33]:
dec = pd.concat([dec_dt_mp, dec_bpw_gp], ignore_index=True)

In [41]:
for crime in sorted(dec['nibrs_code'].unique()):
    print(crime)

09A
09C
120
13A
13B
13C
200
220
23D
23F
23G
23H
240
280
290
35A
35B
520
90C
90J
90Z


In [42]:
nibrs_codes_dict = {'Group A': {},  'Group B': {}}\
group_a = {'Person'}
nibrs_codes_dict['Group A'][]

{}